# Enunciado


Este trabalho usa SageMath nas suas implementações


1. Pretende-se construir em torno de uma cifra assimétrica um conjunto de técnicas criptográficas destinadas a fins distintos. Apesar de todas as alíneas do problema poderem ser  respondidas com a maioria das cifras assimétricas clássicas ou pós-quânticas, neste problema vamos exemplificar o processo com uma técnica simples da família Diffie-Hellman nomeadamente a cifra assimétrica ElGamal com parâmetros de segurança $\,\lambda\,$.
    1. Implemente um esquema  PKE $\,\mathsf{ElGamal}(\lambda)\,$ (ver Capítulo 4) num subgrupo de ordem prima $\,q\,$,  com $\,|q|\geq \lambda\,$, do grupo multiplicativo $\,\mathbb{F}^\ast_p\,$ com $\,p\,$ um primo que verifica $\,|p| \geq \lambda\times|\lambda|$ . Identifique o gerador de chaves e os algoritmos de cifra de decifra neste esquema. Identifique o núcleo deterministico do algoritmo de cifra.
    2. Supondo que a cifra que implementou é IND-CPA segura (de novo Capítulo 4), usando a transformação de Fujisaki-Okamoto implemente um PKE que seja IND-CCA seguro.
    3. A partir de (b) construa um esquema de KEM que seja IND-CCA seguro.
    4. A partir de (b) construa uma implementação de um protocolo autenticado de "Oblivious Transfer" $\,\kappa$-out-of-$n\,$.

# **Exercício 1.a.**

EL GAMAL
Qualquer PKE é determinado por três algoritmos: geração de chaves, cifra e decifra:

 ----------<br><br>$\text{GenKeys}(\lambda)$                                 …       $\lambda\;$ é o parâmetro de segurança<br><br><br>    - gerar aleatoriamente um primo $\,q \approx 2^\lambda$                                      <br>    - gerar um primo $p$  tal que  $\,\mathbb{F}_p^\ast\,$ tem um sub-grupo de ordem $\,q\,$ ; calcular um gerador $g$ desse sub-grupo<br>    - gerar aleatoriamente  $\,0 <s < q\,$ ,  a chave privada<br>    - calcular e  revelar  a chave pública   $\,\mathsf{pk} \equiv \langle p,q, g,g^s\rangle$<br>----------<br><br>$\text{Enc}(\mathsf{pk},m)$                                   …   a mensagem $m$ é um elemento de $\mathbb{F}_p^\ast$ <br><br><br>    - obter elementos públicos  $\,p,q,g,g^s \,\gets\,\mathsf{pk}$<br>    - gerar aleatoriamente  $\,0 <\omega < q$ <br>    - calcular  $\,\gamma \gets g^\omega\;$ e $\,\kappa \gets (g^s)^\omega\,$.<br>    - construir  o criptograma $\,\mathbf{c}\gets \langle\,\gamma\,,\, m\times\kappa\,\rangle\,$<br>----------<br><br>Note-se que se verifica $\,\kappa = \gamma^s\,$.<br> 
----------<br><br>$\text{Dec}(\mathsf{sk},\mathbf{c})$  …  $\mathsf{sk} = s$ é a chave privada<br><br>
- obter a chave privada $s$<br>    
- obter o criptograma $\mathbf{c} = \langle \gamma, \delta \rangle$<br>    
- calcular $\kappa \gets \gamma^s \mod p$<br>    
- calcular $\kappa^{-1} \mod p$<br>    
- recuperar a mensagem original: $m \gets \delta \times \kappa^{-1} \mod p$<br>    

In [1]:
print_tentativas_genKeys = False

In [2]:
from sage.all import *
lambda_security = 128  # Define um tamanho de bits para q

Para encontrar um gerador do grupo multiplicativo temos que encontrar um número cuja ordem seja igual à do grupo, i.e.:
- Dada a ordem do grupo multiplicativo $F_p^* = \phi(p)$ o gerador g tem de ter ordem $\phi(p)$, ou seja, $g^{\phi(n)}=1 \ mod \ p$

In [3]:
def find_generator(p):
    """Encontra um gerador do grupo multiplicativo F_p^*."""
    if not is_prime(p):
        raise Exception("O p de input não é primo")

    phi_p = p - 1  # Para p primo, phi(p) = p - 1

    fatoracao = factor(phi_p)
    fatores_primos = list(set([q for q, e in fatoracao]))
    
    #print(f"Fatoração de phi(p): {fatoracao}")
    #print(f"Fatores primos de phi(p): {fatores_primos}")

    # Itera sobre possíveis geradores
    for g in range(2, p):
        if gcd(g, p) != 1:
            continue  # Ignora números não coprimos com p

        is_gerador = True
        for q in fatores_primos:
            if pow(g, phi_p // q, p) == 1:
                is_gerador = False
                break

        if is_gerador:
            return g 

    return None  

In [4]:
def gen_keys(lambda_security):
    print("----------------------------------Geração de chaves:--------------------------------------")
    # Gerar aleatoriamente q com bit_length maior que lambda
    q = random_prime(2^129 - 1, False, 2^128)
    print("Parâmetro q gerado:",q)
    
    #Gera-se sucessivamente inteiros pi = q*2^i+1 até que pi seja um primo suficientemente grande, ou seja |p| > 1024
    i = 1
    p_i = q * (2^i) + 1 
    tamanho_desejado = lambda_security * lambda_security.nbits()
    
    while True:
        # Caso o p_i não convergir tenta um novo q
        if p_i.nbits() > 2500:
            q = random_prime(2^129 - 1, False, 2^128)
            i = 1
            p_i = q * (2^i) + 1
            print("p não convergiu, novo valor de q: ",q)
            
        if is_prime(p_i) and p_i.nbits() >= tamanho_desejado:
            break  
    
        i += 1
        p_i = q * (2^i) + 1
        if print_tentativas_genKeys:
            print(f"Tentativa {i}: p_i = {p_i} (Tamanho: {p_i.nbits()} bits)")

    p = p_i
    print("Parâmetro p gerado:",p)
    
    # Fp é um grupo multiplicativo se para todo o x pertencente a Fp gcd(x,p) = 1 (trivial já que p é primo)
    # Criar o corpo finito F_p
    F_p = GF(p)
    # O grupo multiplicativo F_p^* é o conjunto de elementos não nulos de F_p
    F_p_star = F_p.unit_group()
        
    # Agora obtemos um gerador do subgrupo de ordem q
    g = find_generator(p)
    print("gerador (g):",g)
    
    # Gerar aleatoriamente a chave privada 0<s<q
    s = randint(1, q-1)
    print("chave privada (s):", s)
    print("---------------------------------------------------------------------------------")
    return (p, q, g, pow(g, s, p)), s

## Escolha de p:
A ordem desse grupo é $\,n = p - 1\,$ e para que o DLP seja  complexo não basta apenas que $\,p\,$ seja grande: é também necessário que o maior factor primo de $\,(p-1)\,$ seja também grande. 
Para garantir estas condições o primo $\,p\,$  é gerado de uma determinada forma:

1. Gera-se um primo $\,q\,$ grande: com mais de $\lambda$ bits de tamanho; este vai ser o maior factor de $\,(p-1)\,$
2. Gera-se sucessivamente inteiros  $\,p_i\;=\;q\,2^i + 1\,$ até que $\,p_i\,$ seja  um primo suficientemente grande .

## Como é que $F_p^*$ tem um subgrupo de ordem q? (Teorema de Lagrange)
O Teorema de Lagrange diz que, se um grupo G tem ordem finita e H é um subgrupo de G, então |H| é um divisor de |G|.

Visto que $F_p^*$ tem ordem $p-1$ e $p-1$ divide $2q$ (e consequentemente $q$) então podemos afirmar que $F_p^*$ tem um subgrupo de ordem q. 

------------------

In [5]:
def enc(pk, m):
    p, q, g, g_s = pk
    omega = randint(1, q-1)  # Escolhe um valor aleatório ω entre 0 e q
    gamma = pow(g, omega, p)  # γ = g^ω mod p
    kappa = pow(g_s, omega, p)  # κ = (g^s)^ω mod p
    c = (gamma, (m * kappa) % p) 
    return c

------------------

In [6]:
def dec(sk, pk, c):
    p, _, _, _ = pk
    gamma, delta = c
    kappa = pow(gamma, sk, p)  # κ = γ^s mod p
    kappa_inv = inverse_mod(Integer(kappa), Integer(p))  # Calcula o inverso de κ mod p
    m = (delta * kappa_inv) % p  # Recupera a mensagem original
    return m

------------------

In [7]:
# Exemplo
pk, sk = gen_keys(lambda_security)  # Geração de chaves
m = randint(1, pk[0]-1)  # Mensagem aleatória em F_p*
c = enc(pk, m)  # Cifra a mensagem
m_dec = dec(sk, pk, c)  # Decifra a mensagem

# Exibir os resultados
print(f"Mensagem original: {m}")
print(f"Criptograma: {c}")
print(f"Mensagem decifrada: {m_dec}")
print(f"Decifração bem-sucedida? {m == m_dec}")
print("---------------------------------------------------------------------------------")

----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 531954973819781813540864378852366836303
Parâmetro p gerado: 2640033387246496528358028313590237472201593100791601078809484793201534817935282678575342713073589419576162841169782658568646568176939383113722685043327226679144609073179311797766320226700746899250013440474681180800101858680512450821115804415589486788480598977949978065286415300908665460225177010920073634555491531756506756857500541020534004348982658090174849951244113504435946234982607556276581934347356852822508306433
gerador (g): 3
chave privada (s): 520250154583031319975560258422362935375
---------------------------------------------------------------------------------
Mensagem original: 1041039525159970521386620925441569163137944286284821891665109277418310129755845983670661385234336091849940571187073209895367083809305182791107718856574551788396989531723511073170031702487634439807536063399102717328361892493360969003199

--------

# **Exercício 1.b.**

Supondo que a cifra que implementou é IND-CPA segura (de novo Capítulo 4), usando a transformação de Fujisaki-Okamoto implemente um PKE que seja IND-CCA seguro.

### **Transformar um  PKE-IND-CPA em um PKE-IND-CCA**

A transformação FO original constrói, a partir de $\,(E_p,D_s)\,$,  um novo esquema de cifra assimétrica $\,(E'_p,D'_s)\,$ , usando um  “hash” pseudo-aleatório $\,h\,$ de tamanho $\,\lambda\,$ e um “hash” pseudo-aleatório $\,g\,$ de tamanho $\,|x|\,$.

O algoritmo de cifra parametrizado pelos dois “hashs”  $\,h,g\,$    é 

  $$E'_{p}(x)\;\equiv\;\vartheta\,r\gets \{0,1\}^\lambda\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\vartheta\,c\gets f_p(r,r') \,\centerdot\, (y\,,\,c)$$

O algoritmo $\,D'_{s}\,$ rejeita o criptograma se detecta algum sinal de fraude. 


$$D'_{s}(y,c)\;\equiv\;\vartheta\,r \gets D_s(c)\,\centerdot\,\vartheta\,r'\gets h(r,y)\,\centerdot\,\mathsf{if}\;\;c\neq f_p(r,r')\;\;\mathsf{then}\;\;\bot\;\mathsf{else}\;\;y\oplus g(r)$$

### Definições de variáveis

In [8]:
print("----------------------------------Definição de veriáveis----------------------------------")
lambda_bits = 128

pk, sk = gen_keys(lambda_bits)
x = randint(1, pk[0]-1)  # Mensagem aleatória em F_p*
length_in_bytes = (x.bit_length() + 7) // 8
x = x.to_bytes(length_in_bytes, byteorder='big')
print("length x:",len(x))
print("Mensagem de input:",x)
print("-------------------------------------------------------------------------------------------")

----------------------------------Definição de veriáveis----------------------------------
----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 539338107706871179792608347208784587923
Parâmetro p gerado: 569858745469196865310420302837889024816587330085103183588459863188349013621952581701588434075736120307478274854429730973700827366327424187624349876351993120614586880784615696254972871083029215251453853696462302742673903196447403615865536930329762592030887911033506902735505444373456878554660275954801239392257
gerador (g): 3
chave privada (s): 25745406858872374952140567909592436881
---------------------------------------------------------------------------------
length x: 129
Mensagem de input: b"\x02\x15\x07\x93\x95\x85\x92\xdd\xc0\xf9\xec\xa0\x1c\x18\x1a\xc5\xb34\x8c{\xbc#\xc0\xb6L\r\xd5X\x84\xecV#\x80\x92\xa6\xfc\x88'~*q\xdf/\x9b\xe5wO\xc3\xad\xf0\xba`\xec\x8f\xc1\xe8\x829\xed\xd4\n\xb6Y\xc4\xf0\xe8\x02Iq'a=\x12\x92\xe6\xe5\x13

-------

### Funções auxiliares:
- $g$, um "hash" pseudo-aleatório de tamanho $|x|$
- $h$, um "hash" pseudo-aleatório de tamanho $lambda$
- $f_p$ o núcleo determinístico da cifra ElGamal

In [9]:
import hashlib

def g(r):
    """Hash pseudoaleatório g(r) com tamanho igual ao da mensagem x"""
    g = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    g.update(r_bytes)
    final_hash = g.digest()  # Truncar para o tamanho de x
    while len(final_hash) < len(x):
        g = hashlib.sha512()
        g.update(r_bytes)
        final_hash += g.digest()
    #print("g(r) hash:", final_hash[:len(x)])
    return final_hash[:len(x)]

def h(r, y):
    """Hash pseudoaleatório h(r, y) com tamanho lambda_bits"""
    h = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    ry = bytes(a ^^ b for a, b in zip(r_bytes, y))
    h.update(ry)
    full_hash = h.digest()[:lambda_bits // 8]  # Truncar para lambda bits
    #print("h(r, y) hash:", full_hash)
    return full_hash

#Núcleo determinístico da função enc anterior
def f_p(pk, r, rlinha):
    p, q, g, g_s = pk
    gamma = pow(g, rlinha, p)  # γ = g^ω mod p
    kappa = pow(g_s, rlinha, p)  # κ = (g^s)^ω mod p
    return (gamma, (r * kappa) % p)

### Cifra IND-CPA segura 

In [10]:
print("----------------------------------CIFRA----------------------------------")
def enc_fujisaki(pk,x):
    r = ZZ.random_element(2^(lambda_bits - 1), 2^lambda_bits)
    y = bytes(a ^^ b for a, b in zip(x, g(r)))
    rlinha = h(r,y)
    c = f_p(pk,r,int.from_bytes(rlinha,"big"))
    return (y,c)

(y,c) = enc_fujisaki(pk,x)
print("(y,c) = ",(y,c))
print("-------------------------------------------------------------------------------------------")

----------------------------------CIFRA----------------------------------
(y,c) =  (b'\xa7\xbe_\x8bm\xa7\x0f\xdd\xfa\xa3,\x0f|=\x8e\x1a\x88\xf0fF\x7f=C\x8e\xf8q\xb7\xdc;a\xd9\x98&n\xfbZ\x8f\xfaR\xc8\xf3\xa7\r\xb0\xdf\xa5,\x97\x1b\x11\xc3\xd8"\xc7\x87\x06\x96\xf8\xdf\xac\xbc\xce\xb1}UCZQ\x89\x05\xfc=(\xc8&Js\x06\x9f\'2\xfa\xffhw\xf6X\xcf{\xbe\xdd\x8a\x0eC7\x8b2\xc6\xban\x98\xf5\x8b\xb8]\xafH\x9e-\x16<H\x8e\x08\x87gGvC\xc8\x85\x95:|`{|\xed3', (113578045274212432069816488953812817433540476992133139298717455444177999609316401945845027779583888127648015325877679876861777773126326077646559534787150602094925538569460218127775351307498533854881887884869238214377108300414704877534811214181528750917134425544795229333600669481095845129053383840853099158028, 111206749837682166042319034242764252547486024671867614717256158532646992243274967551512406462295445461177058915779258756243594978349860933609843792949639179619237580768336114377577843428641197802459749372105524550105491285497232902729245296363

### Decifra

In [11]:
print("----------------------------------------DECIFRA--------------------------------------------")
def dec_fujisaki(sk,pk,y,c):
    r = dec(sk,pk,c)
    rlinha = h(r,y)
    if c != f_p(pk,r,int.from_bytes(rlinha,"big")):
        raise Exception("O criptograma não corresponde a fp(r,r'), absurdo")
    else:
        res = bytes(a ^^ b for a, b in zip(y, g(r)))
        return res

print("Mensagem decifrada: ",dec_fujisaki(sk,pk,y,c))
print("x == dec_fujisaki(sk,pk,y,c)?: ",x == dec_fujisaki(sk,pk,y,c)) 
print("-------------------------------------------------------------------------------------------")

----------------------------------------DECIFRA--------------------------------------------
Mensagem decifrada:  b"\x02\x15\x07\x93\x95\x85\x92\xdd\xc0\xf9\xec\xa0\x1c\x18\x1a\xc5\xb34\x8c{\xbc#\xc0\xb6L\r\xd5X\x84\xecV#\x80\x92\xa6\xfc\x88'~*q\xdf/\x9b\xe5wO\xc3\xad\xf0\xba`\xec\x8f\xc1\xe8\x829\xed\xd4\n\xb6Y\xc4\xf0\xe8\x02Iq'a=\x12\x92\xe6\xe5\x13#\x0b\xf8\t>\x15U\xb4\xe8\xdb\xf7\xcf\xc2\xbf\x0e\xb1\xce\xb80\x94:\xe7\xc8\x9f(\xa7Z\xdf\xd7j\xb5\x17\xc4_\x1c8\xe9\xfe\xdf\x89>\x05&\x91T\x08\x04\xd6\x03\x94T\x96"
x == dec_fujisaki(sk,pk,y,c)?:  True
-------------------------------------------------------------------------------------------


# **Exercício 1.c.**

Associado a um mecanismo de encapsulamento de chaves existe um algoritmo de revelação ou KRev (“_key revelation_”), que a partir do encapsulamento de uma chave, revela-a.  Naturalmente, por analogia com as cifras assimétricas, o KEM é um algoritmo  público, total e probabilístico,  enquanto que o KRev é um algoritmo privado, parcial e determinístico.

A condição de correção será
                                            $$(e,k)\,\gets\,\mathsf{KEM}\quad\;\text{sse}\;\quad \mathsf{KRev}(e)\,\simeq\, k$$

Combinando o exercício anterior com um KEM o esquema pode-se descrever pelo par de algoritmos:

$$\left\{\begin{array}{lcl}E(x) & \equiv & \vartheta\,(e,k)\gets\mathsf{KEM}\,\centerdot\,(e\,,\,k \oplus x) \\ D(e,c) & \equiv & \vartheta\,k \gets \mathsf{KRev}(e)\,\centerdot\,k\oplus c\end{array}\right.$$

Substituindo KEM pela função de cifra do exercício (b) obtemos uma segurança IND-CCA para a encapsulação da chave.
Como o KEM é um mecanismo apenas de encapsulamento da chave iremos utilizar em conjunto com um DEM (neste caso OTP) para demonstrar que resulta para cifrar uma mensagem, como é demonstrado no diagrama:

![PKE standard KEM + OTP](KEMDEM.png)

----------

### Definições de variáveis

In [12]:
lambda_bits = 128

pk, sk = gen_keys(lambda_bits)
x = randint(1, pk[0]-1)  # Mensagem aleatória em F_p*
length_in_bytes = (x.bit_length() + 7) // 8
x = x.to_bytes(length_in_bytes, byteorder='big')

print("Mensagem aleatória em F_p*: ",x)

----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 609379455678639404745547312236057769697
p não convergiu, novo valor de q:  567482204201020615387549895344044059069
Parâmetro p gerado: 48644965298114282994770203372489044685124057278048494410036532220542663143730268376589310390947635250071425872781513314351175836713855405752230039449480665404119706670397674306106306177912946200941709670540668458589655040347901169930967664671955446757952859764630343166858994219029313976917988105856651086317755962990568115741456029871767553
gerador (g): 3
chave privada (s): 131131757256576817243789205443369027496
---------------------------------------------------------------------------------
Mensagem aleatória em F_p*:  b"\x04y<\xf1\x1f\xf8_\xa3\x98XjC\xd9&\xcb(\x0e@\x85\x0eAM7e\x8b\xed\xb8s\xd2Fn]\xb1Y\x0b\x94\x15\xa6\xf8=t\xd9\x8c#t\xd8km\x8c\x9ap \x84\x9c1\xae\x006\xa7\xc3\xa6\x1ad\x11m\xe0XDY\xb2p9z\xc4\xd1\xdc3vi\xacC\xf9\xd4hq*\xe1w\xafV

-------

### KEM + DEM Encrypt

Construir algoritmo de cifra tal que
$ E(x) \equiv \vartheta (e,k) ← KEM⋅(e,k⊕x)  $ 
onde 
- $k$ é gerado aleatoriamente;
- $e$ é o encapsulamento dessa chave.

In [13]:
def kem_encrypt(pk, x):
    # Gerar chave simétrica aleatória k
    k = ZZ.random_element(2^(8*len(x)-1), 2^(8*len(x))).to_bytes(len(x), 'big')
    print("k: ",k)
    # Utilizar cifra do ex1.b. como KEM para encapsular k
    e = enc_fujisaki(pk,k)
    
    # Aplicar DEM (neste caso OTP) da chave gerada k com a mensagem x
    ciphertext = dem(x,k)

    return e, ciphertext

def dem(x,k):
    ciphertext = bytes(a ^^ b for a, b in zip(x, k))
    while len(ciphertext) < len(x):
        ciphertext += bytes(a ^^ b for a, b in zip(x[len(ciphertext):], k))
        ciphertext[:len(x)]
    return ciphertext

-------

### KEM + DEM Decrypt

A decifração é dada por:
$ D(e,c) ≡ ϑk←KRev(e)⋅k⊕c $
onde 
- $KRev$ no nosso caso é a técnica de decifração utilizada no exercício 1.b.;
- $e$ é o encpasulamento da chave gerada;
- $c$ o criptograma recebido de KEM.

In [14]:
def kem_dem_decrypt(sk, pk, e, ciphertext):
    (y, c) = e
    k = dec_fujisaki(sk, pk, y, c)
    print("k: ",k)
    x = dem(ciphertext,k)
    return x

---------

### Teste

In [15]:
print("-------------------------------------------------------------------------------------------------------------------")
print("mensagem (x): ",x)
e, ciphertext = kem_encrypt(pk,x)
print("-------------------------------------------------------------------------------------------------------------------")
print("e: ",e)
print("-------------------------------------------------------------------------------------------------------------------")
print("k (+) x: ",ciphertext)
print("-------------------------------------------------------------------------------------------------------------------")

mensagem = kem_dem_decrypt(sk, pk, e, ciphertext)

print("mensagem (final): ",mensagem)
print("-------------------------------------------------------------------------------------------------------------------")
print("mensagem inicial == mensagem final ?: ",x == mensagem)
print("-------------------------------------------------------------------------------------------------------------------")

-------------------------------------------------------------------------------------------------------------------
mensagem (x):  b"\x04y<\xf1\x1f\xf8_\xa3\x98XjC\xd9&\xcb(\x0e@\x85\x0eAM7e\x8b\xed\xb8s\xd2Fn]\xb1Y\x0b\x94\x15\xa6\xf8=t\xd9\x8c#t\xd8km\x8c\x9ap \x84\x9c1\xae\x006\xa7\xc3\xa6\x1ad\x11m\xe0XDY\xb2p9z\xc4\xd1\xdc3vi\xacC\xf9\xd4hq*\xe1w\xafV\xf2\x81Ui\xaa\xec\xb2\x18\x8c7'\xf4cl\r\x94W\xfe@\xcd\xac\xa3\x1b\x0fo\x80\xc3\x04} \xbe^\x0c\xe5\xc0\x14T\x16t\x98\x83\xf5\x00\xa4\xe3\x95cq\x93\x08\r "
k:  b'\x92l9>\x13\xf6\xcf\xe6P\xbd\x18\x15\x81y\x02\xfe\xd5\x8boW\xeem\x87\xa3t\x11\xb2\x0e1\x10,\x86\x90w\xe6(\xf4\x9e\xf9f\xedd\x92\xf2\x0b\xfb\x86H\xffO~\xf6\xd3/\xefo\xd6\x84\x12eX$\xff7\xa80_cx\xd1__\xa7;\xbbY\x1c\xff#*\x99UE\x0f`L\xd2\x98\xde\\\x1c\xbb\x1886O\xb1\x89\xdd\xf9\xb6\xe2\x82\xfd\x93\x94\xbb\xef\xf8gv%CcY\xeaF\xcf\xe2U\xcb\xfe\xf0\xa0\xd5\x0f\xd9\x1f\x0e\x85S\xc4\xb7\x99\xcc@\x03\x91\x8f\xe2\xe8\x89'
------------------------------------------------------------------

----------------

# Exercício 1.d.


O protocolo de “oblivious transfer”  implementa um mecanismo de transferência de informação entre dois agentes: o **Provider** (também designado por Sender) e o **Receiver** (também designado por Adversário) . Em linhas gerais, o protocolo caracteriza-se da forma seguinte:


1. O **Provider** põe à disposição para comunicação futura $\,n\,$ items de informação (ou mensagens) que ele enumera como $\,m_1, m_2, \cdots,m_{n}\;$ e que armazena de forma privada. 
    Nesta fase a única informação tornada pública é o número  de mensagens  $\,n$ .
2. O Receiver informa o **Provider** que pretende receber $\,\kappa\,$ das $\,n\,$ mensagens
3. Caso o **Provider** aceite o par $(n,\kappa)\;$ os dois agentes, a começar pelo **Provider**, trocam uma sequência de mensagens e, no final,
    1. O **Receiver** passa a conhecer exatamente  $\,\kappa\,$ mensagens mas continua a ignorar o conteúdo de todas as restantes $\,n-\kappa\,$ mensagens.
    2. O **Provider** ignora a identificação (“is oblivious of”)  das $\,\kappa\,$ mensagens que o Receiver passou a conhecer.

O protocolo usa um esquema PKE   $\,\{(E_p,D_s)\}_{(s,p)\in\mathcal{G}}$ que neste caso irão ser a cifra e decifra do exerício 1.b.. 

----

### Criterion

O critério $C{κ,n}$ define quais vetores de chaves públicas **p** são considerados válidos. O **Receiver** deve criar um vetor **p** onde:

- Algumas entradas são chaves públicas "boas" (as que correspondem às mensagens que quer receber).
- O resto das entradas são chaves públicas "más" (geradas para manter a segurança do protocolo).

O critério $\,\mathcal{C}_\kappa\,$ depende da estrutura algébrica usada pelo mecanismo de geração de pares de chaves. Vamos considerar:

- Cada chave pública, válida ou não, é codificada por um inteiro em $\,\mathbb{F}_p^*\,$.
- Uma matriz de “rank” completo  $\,\mathsf{A} \in \mathbb{F}_p^{* n\times(n-\kappa)}\,$ e um vector $\,\mathsf{u}\neq 0\in \mathbb{Z}_q^{n-\kappa}\,$ ; estes elementos são gerados por um XOF a partir de uma “seed”   $\,\rho\,$ . A  “seed” é aleatoriamente gerada e os restantes elementos são construídos com o XOF, por tentativas, até se verificarem as condições exigidas.
- O critério tem a forma de um sistema de equações lineares  $\;\mathsf{p} \times \mathsf{A}\,=\,\mathsf{u}$ .

In [16]:
import numpy as np

class CknCriterion:
    def __init__(self, kappa, n, q):
        self.kappa = kappa
        self.n = n
        self.q = int(q) 
        self.seed = np.random.randint(0, 2**32)
        self.A = self.generate_A()
        self.u = self.generate_u()
        self.Fp = GF(q).unit_group()
    
    def generate_A(self):
        """Gera a matriz A usando XOF a partir da seed"""
        np.random.seed(self.seed)
        A = random_matrix(GF(self.q), self.n, self.n - self.kappa)
        return A
    
    def generate_u(self):
        """Gera o vetor u, que deve ser não nulo"""
        np.random.seed(self.seed + 1) 
        u = vector(GF(self.q), [randint(1, self.q - 1) for _ in range(self.n - self.kappa)])
        return u
    
    def verify(self, p):
        """Verifica se p satisfaz o critério Ckn, ou seja, se p * A = u"""
        if len(p) != self.n:
            raise ValueError(f"p deve ter {self.n} elementos")
        
        p_values = [x[0] if isinstance(x, tuple) else x for x in p]
        print("p_values:",p_values)
        
        # Converter p_values para um vetor no corpo finito Z_q
        Zq = GF(self.q)
        p_vector = vector(Zq, p_values)
    
        # Calcular p * A no corpo finito Z_q
        A_matrix = matrix(Zq, self.A) 
        pA = p_vector * A_matrix 
    
        # Verificar se pA é igual a u
        u_vector = vector(Zq, self.u)  # Converter u para um vetor no corpo finito Z_q
        return pA == u_vector
    
    def print_criterion(self):
        print(f"Matriz A:\n{self.A}")
        print(f"Vetor u:\n{self.u}")

In [17]:
class Provider:
    def __init__(self, pk, sk, n_mensagens):
        self.pk,self.sk = gen_keys(lambda_bits) #(p, q, g, pow(g, s, p)) , s
        self.numero_de_mensagens = n_mensagens
        # Informação privada das mensagens:
        self.messages = [randint(1, self.pk[0] - 1) for _ in range(n_mensagens)]
        self.criterion = None
    def define_criterion(self,kappa):
        n = self.numero_de_mensagens
        q = pk[1]
        self.criterion = CknCriterion(kappa, n, q)

In [18]:
import hashlib
from random import sample

class Receiver:
    def __init__(self, k, n_mensagens, q):
        self.k = k  # Número de chaves privadas geradas
        self.n_mensagens = n_mensagens
        self.q = q 
        self.I = sample(range(n_mensagens), k)  # Seleção aleatória de k índices
        self.e = self.enumeration(self.I)  
        self.p,self.s_values = self.generate_keys()  
        self.s = self.generate_secret()
        self.tau = self.generate_authentication_tag()

    def enumeration(self, I):
        """Cria a função de enumeração que mapeia {1, 2, ..., κ} para os elementos de I ordenados"""
        I_sorted = sorted(I)
        return {i + 1: I_sorted[i] for i in range(len(I_sorted))}

    def generate_secret(self):
        """Gera um segredo aleatório (simulado como um número grande)"""
        return ZZ.random_element(2**32)

    def generate_keys(self):
        """Gera κ chaves privadas e publicas usando o genkeys de ElGamal"""
        vetor_pk = [0] * self.n_mensagens 
        vetor_sk = []
        for i in range(1,self.k+1):
            pk,sk = gen_keys(lambda_bits) # (p, q, g, pow(g, s, p)), s
            vetor_sk.append(sk)
            vetor_pk[self.e[i]] = pk
        return vetor_pk,vetor_sk

    def generate_authentication_tag(self):
        """Gera a tag de autenticação hash(I, s)"""
        data = str(self.I) + str(self.s)
        return hashlib.sha256(data.encode()).digest()

    def complete_p_vector(self, A, u):
        """Completa o vetor p para satisfazer p * A = u no corpo finito Z_q"""
        Zq = GF(self.q)  # Define o corpo finito Z_q
    
        # Identificar os índices já preenchidos (valores diferentes de 0)
        filled_indices = [i for i in range(self.n_mensagens) if self.p[i] != 0]
        filled_values = vector(Zq, [self.p[i][0] if isinstance(self.p[i], tuple) else self.p[i] for i in filled_indices])
    
        # Criar a matriz A_filled (linhas correspondentes aos índices preenchidos)
        A_filled = matrix(Zq, [A[i] for i in filled_indices])
    
        # Criar a matriz A_empty (linhas correspondentes aos índices vazios, ou seja, onde p[i] == 0)
        A_empty = matrix(Zq, [A[i] for i in range(A.nrows()) if i not in filled_indices])
    
        # Calcular u' = u - (filled_values * A_filled)
        u_prime = vector(Zq, u) - filled_values * A_filled
    
        # Resolver o sistema linear A_empty^T * p_empty = u' no corpo finito Z_q
        try:
            A_empty_T = A_empty.transpose()
            p_empty = A_empty_T.solve_right(u_prime)
        except:
            # Se o sistema for singular, tentar solução alternativa (ex: mínimos quadrados)
            p_empty = A_empty_T.pseudoinverse() * u_prime
    
        # Preencher os elementos desconhecidos no vetor p (apenas onde p[i] == 0)
        empty_indices = [i for i in range(self.n_mensagens) if self.p[i] == 0]
        for i, idx in enumerate(empty_indices):
            # Gerar a chave "má"
            p_mau, q_mau, g_mau, gs_mau = self.gen_mau_keys() 
            self.p[idx] = (p_mau, q_mau, g_mau, gs_mau)
        return self.p
    
    def gen_mau_keys(self, max_attempts=100):
        """Gera uma chave 'má' no formato (p, q, g, g^s)"""
        attempts = 0
        while attempts < max_attempts:
            # Gerar q_mau como um número primo aleatório
            q_mau = random_prime(2^128, False, 2^127)
    
            # Tentar encontrar um k tal que p_mau = q_mau * 2^k + 1 seja primo
            for k in range(1, 10):  # Limitar k a um intervalo razoável
                p_mau = q_mau * (2^k) + 1
                if is_prime(p_mau):
                    # Criar o corpo finito F_p_mau
                    F_p_mau = GF(p_mau)
    
                    # Encontrar um gerador g_mau do grupo multiplicativo F_p_mau^*
                    g_mau = F_p_mau.multiplicative_generator()
    
                    # Gerar a chave privada s_mau aleatoriamente
                    s_mau = randint(1, q_mau - 1)
    
                    # Calcular g^s mod p
                    gs_mau = pow(g_mau, s_mau, p_mau)
    
                    return p_mau, q_mau, g_mau, gs_mau
    
            attempts += 1
    
        raise ValueError(f"Não foi possível gerar p_mau após {max_attempts} tentativas.")
    
    def print_info(self):
        print("----------------------------------------------------------------------")
        print(f"Seleção I: {self.I}")
        print("----------------------------------------------------------------------")
        print(f"Função de enumeração e: {self.e}")
        print("----------------------------------------------------------------------")
        print(f"Segredo s: {self.s}")
        print("----------------------------------------------------------------------")
        print(f"Chaves privadas s_i: {self.s_values}")
        print("----------------------------------------------------------------------")
        print(f"Vetor p (com chaves públicas mapeadas): {self.p}")
        print("----------------------------------------------------------------------")
        print(f"Tag de autenticação τ: {self.tau}")
        print("----------------------------------------------------------------------")

## Provider

Inicialmente:
- Gera chaves
- Gera mensagens
- Expõe apenas o número de mensagens, matriz $A$ e vetor $u$

In [19]:
n_mensagens = 100
provider = Provider(None,None,n_mensagens)

----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 414590764414933417545568386157582998973
Parâmetro p gerado: 14354090131440552994945107926766013714394188703392476131012522558284486258795702048349874552513523103453051632957477599294155916478081785234096169785671124371496101436136726115821707494594845008624412540590560550582432429148669931325618473762456026155931230413689197947786305695272838217983218817814034101628305409
gerador (g): 3
chave privada (s): 368446863676237977065668789005509841702
---------------------------------------------------------------------------------


## Receiver

Escolhe o número de mensagens que pretende receber do Provider tendo acesso ao número de mensagens disponíveis

In [20]:
k = 60
receiver = Receiver(k,n_mensagens,provider.pk[1]) # Receiver escolhe o conjunto I já na sua construção

----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 520551225695642813209194840008132718263
p não convergiu, novo valor de q:  603275853292820270902229293475194444727
Parâmetro p gerado: 202004771659655246425480962907746731652771861232440020221403333286306538524764058200972421408051565526700181442665046792233704956746326748898448334124423600744173207955168716705072005822912942131469111784557507965201605586179572334718494540472542272267290751567188101609129169118922464729488280084792746625635580290773455581677881326043137
gerador (g): 3
chave privada (s): 248492468099137661529400966240921513341
---------------------------------------------------------------------------------
----------------------------------Geração de chaves:--------------------------------------
Parâmetro q gerado: 674089667042639778370330252298254999561
p não convergiu, novo valor de q:  547635889938795306079221559269285254053
Parâmetro p gerado: 224815781605

### 1. O **Provider** gera o critério C_{k,n} e envia-o ao **Receiver**

In [21]:
provider.define_criterion(k)

### 2. O Receiver  escolhe um conjunto $\,I \subset \{1,n\}\,$ , de tamanho $\,\#I = \kappa\,$ , que identifica os índices das mensagens que pretende recolher.  
Seja $\;e\;$ a enumeração de $\,I\,$:  a função crescente $\,e\colon\{1,\kappa\}\to \{1,n\}\;$  cuja imagem é $\,I\,$.

O Receiver compromete-se com a escolha de mensagens da seguinte forma (dado o conjunto $I$ e a função crescente $e$):

1. Gera aleatoriamente um segredo $\,\mathbf{s}\,$ e , usando um XOF com$\,\mathbf{s}\,$ como “seed”, constrói$\,\kappa\,$  chaves privadas$\,s_1,\cdots,s_\kappa\,$ .
2. Para cada$\,i\in \{1,\kappa\}\,$ gera chaves públicas $\;\upsilon_i \gets \mathsf{pk}(s_i)\,$ e atribui o valor$\,\upsilon_i\,$ à componente de ordem$\,e(i)\,$ do vector $\,\mathsf{p}\;$; ou seja , executa$\,$ $\,\mathsf{p}_{e(i)} \gets \upsilon_i\,$
3. Gera uma “tag” de autenticação$\,$para a seleção$\,I\,$ e o segredo$\,\mathbf{s}\,$

$$\,\tau \gets \mathsf{hash}(I\,,\,\mathbf{s})\,$$

In [22]:
receiver.print_info()

----------------------------------------------------------------------
Seleção I: [30, 88, 74, 19, 18, 97, 61, 50, 15, 32, 55, 77, 76, 14, 31, 2, 89, 93, 51, 34, 96, 13, 6, 12, 95, 38, 81, 40, 56, 0, 86, 82, 33, 36, 83, 99, 80, 25, 58, 10, 78, 90, 22, 35, 5, 7, 44, 17, 65, 16, 59, 49, 8, 91, 57, 70, 75, 45, 52, 94]
----------------------------------------------------------------------
Função de enumeração e: {1: 0, 2: 2, 3: 5, 4: 6, 5: 7, 6: 8, 7: 10, 8: 12, 9: 13, 10: 14, 11: 15, 12: 16, 13: 17, 14: 18, 15: 19, 16: 22, 17: 25, 18: 30, 19: 31, 20: 32, 21: 33, 22: 34, 23: 35, 24: 36, 25: 38, 26: 40, 27: 44, 28: 45, 29: 49, 30: 50, 31: 51, 32: 52, 33: 55, 34: 56, 35: 57, 36: 58, 37: 59, 38: 61, 39: 65, 40: 70, 41: 74, 42: 75, 43: 76, 44: 77, 45: 78, 46: 80, 47: 81, 48: 82, 49: 83, 50: 86, 51: 88, 52: 89, 53: 90, 54: 91, 55: 93, 56: 94, 57: 95, 58: 96, 59: 97, 60: 99}
----------------------------------------------------------------------
Segredo s: 195434846
------------------------------

Em seguida completa a definição de $\,\mathsf{p}\,$ atribuindo às compompentes $\,\{\mathsf{p}_j\}_{j\in\!\!\!/I}$valores tais que o vetor de chaves públicas  $\,\mathsf{p}\,$ seja aceite pelo critério $\,\mathcal{C}_{\kappa,n}\,$.

Para que o vetor **p** satisfaça a equação $p×A=u$, onde **A** é a matriz gerada pelo critério $C_{k,n}$ e **u** é o vetor correspondente, o Receiver precisa preencher os espaços em **p** que não foram preenchidos pelas chaves públicas de forma que a equação seja satisfeita.

Mais concretamente essa forma de completar o vetor é descrita na classe Receiver:

O código realiza a seguinte sequência de operações para completar o vetor p de forma que satisfaça a equação p×A=up×A=u no corpo finito $Z_q$:

- Definição do corpo finito $Z_q$ usando ```GF(self.q)```.

- Identificação dos índices preenchidos no vetor $p$ usando ```filled_indices```, que são obtidos a partir dos valores em ```self.e```. Os valores correspondentes a esses índices são armazenados em ```filled_values```.

- Criação das matriz ```A_filled```  partir das linhas da matriz $A$ correspondentes aos índices preenchidos.

- Criação das matriz ```A_empty``` a partir das linhas da matriz $A$ correspondentes aos índices vazios.

- Cálculo do vetor $u′$ subtraindo a contribuição dos elementos preenchidos de $u$, ou seja, ```u′ = u − (filled_values × A_filled)```.

- Resolução do sistema linear $p_{empty} = A_{empty}^T . u′$ para preencher os valores desconhecidos de $p$ (chaves "más").

- Preenchimento de p, colocando os valores de $p_{empty}$ índices correspondentes no vetor p.

Finalmente o Receiver envia ao Provider a “tag” $\,\tau\,$ e o vetor $\,\mathsf{p}\,$

In [23]:
completed_p = receiver.complete_p_vector(provider.criterion.A, provider.criterion.u)
tau = receiver.tau

### 3. O Provider  determina  $\,\mathcal{C}_{\kappa,n}(\mathsf{p})\,$; se $\,\mathsf{p}\,$ não for aceite pelo critério então aborta o protocolo.

Se $\,\mathsf{p}\,$  for aceite,  então usa  a variante IND-CCA  da cifra IND-CPA com a “tag”  $\tau$

$$E'_{p}(x,\tau)\;\equiv\;\vartheta\,r\gets \{0,1\}^\lambda\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\,\vartheta\,r'\gets h(r,y,\tau)\,\centerdot\,\vartheta\,c\gets f_p(r,r') \,\centerdot\, (y\,,\,c)$$

cuja característica específica é o facto de se incluir o “tag” $\,\tau\,$ no “hash”  $\;h(r,y,\tau)\;$usado para construir a pseudo-aleatoriedade $\,r'\,$.

Usando esta cifra o Provider  constrói $\,n\,$ criptogramas 
$$\;(y_i,c_i) \gets E'_{p_i}(m_i)\quad$$  com $\,i\in\{1,n\}$
que envia para o Receiver.

Iremos definir um novo "hash" pseudo-aleatório $h$ para que consiga receber como argumento a tag $\tau$:

In [24]:
def h_OT(r,y,t):
    """Hash pseudoaleatório h(r, y, t) com tamanho lambda_bits"""
    h = hashlib.sha512()
    r_bytes = r.to_bytes((r.bit_length() + 7) // 8, 'big')
    ry = bytes(a ^^ b for a, b in zip(r_bytes, y))
    ryt = bytes(a ^^ b for a, b in zip(ry, t))
    h.update(ryt)
    full_hash = h.digest()[:lambda_bits // 8]  # Truncar para lambda bits
    #print("h(r, y) hash:", full_hash)
    return full_hash

E definir a cifra, que na realidade irá ser a do exercício 1.b. mas com um argumento extra $\tau$:

In [25]:
def enc_fujisaki_OT(pk,x,t):
    # r gerado aleatoriamente com lambda bits
    r = ZZ.random_element(2^(lambda_bits - 1), 2^lambda_bits)

    # y = x XOR g(r)
    y = bytes(a ^^ b for a, b in zip(x, g(r)))

    # r' = h(r,y,tau)
    rlinha = h_OT(r,y,t)
    print(rlinha)

    # c gerado a partir do núcleo determinístico 
    c = f_p_OT(pk,r,int.from_bytes(rlinha,"big"))

    #Devolver criptograma
    return (y,c)

In [26]:
def f_p_OT(pk, r, rlinha):
    p, q, g, g_s = pk
    # Verificar se p é primo
    if not is_prime(p):
        raise ValueError(f"p = {p} não é um número primo. Não é possível criar o corpo finito GF(p).")


    # Calcular gamma e kappa
    gamma = pow(g, rlinha, p)  # γ = g^ω mod p
    kappa = pow(g_s, rlinha, p)  # κ = (g^s)^ω mod p

    # Multiplicar r_fp por kappa no corpo finito F_p
    result = (r * kappa) % p

    return (gamma, result)


def f_p(pk, r, rlinha):
    p, q, g, g_s = pk
    gamma = pow(g, rlinha, p)  # γ = g^ω mod p
    kappa = pow(g_s, rlinha, p)  # κ = (g^s)^ω mod p
    return (gamma, (r * kappa) % p)

Determinamos então $C_{k,n}(p)$, se este for aceite ciframos todas as mensagens com as chaves públicas fornecidas em $p$:

In [27]:
if not provider.criterion.verify(completed_p):
    print("------------------------------------------------------------------------")
    print("O vetor p foi aceite")
    print("p final:[ ")
    for key in completed_p:
        print(key)
    print("]")
    print("------------------------------------------------------------------------")
    ciphertext_vector = []
    i = 0
    for public_key in completed_p:
        if isinstance(public_key, tuple):
            byte_length = (provider.messages[i].bit_length() + 7) // 8
            ciphertext_vector.append(enc_fujisaki_OT(public_key, provider.messages[i].to_bytes(byte_length, 'big'), tau))
            i += 1
        else:
            raise ValueError(f"encontrada chave pública não primo: {public_key}")
else:
    print("O vetor p não foi aceite. Abortado")

p_values: [202004771659655246425480962907746731652771861232440020221403333286306538524764058200972421408051565526700181442665046792233704956746326748898448334124423600744173207955168716705072005822912942131469111784557507965201605586179572334718494540472542272267290751567188101609129169118922464729488280084792746625635580290773455581677881326043137, 361217068805798759001343690228953973319, 2248157816050213565381545315453015346716464836101355299915112692847958905993965596154442309418399784246923880144407156311760698117678360101233205583331846185829574531401554610082830773255942552732063140500485821107854362361190030943688749756223318444151296733001461866965271543592075718360403173474252569048253740892813573777845294438251650727407404996795466307471471248887968443033075509624833, 2639738244375039209312891689935347852057, 3271640249879569776131335776702135994193, 2700787378582441825094331240624908536514801793204130724660061036703075981714061202357710462621301764718091068773596893409603163

### 4. O Receiver  usa  a variante IND-CCA  da cifra IND-CPA com a “tag” de autenticação $\tau$
$$D'_{s}(y,c,\tau)\;\equiv\;\vartheta\,r \gets D_s(c)\,\centerdot\,\vartheta\,r'\gets h(r,y,\tau)\,\centerdot\,\mathsf{if}\;\;c\neq f_p(r,r')\;\;\mathsf{then}\;\;\bot\;\mathsf{else}\;\;y\oplus g(r)$$
         
uma vez mais a única característica particular deste algoritmo é o uso da “tag” de autenticação $\,\tau\,$ na construção da pseudo-aleatoriedade $\;r'\gets h(r,y,\tau)\;$.


O agente Receiver 
   - conhece, porque criou,  a “tag” $\,\tau\,$que autentica o conjunto de mensagens escolhidas $\,I\,$ e  o respetivo conjunto de chaves públicas (as “boas” chaves).
    - conhece , porque gerou e armazenou num passo anterior,  as chaves privadas $\,s_i\,$ para todos $\,i\in I\,$  
    - conhece, porque recebeu do Receiver,  todos os criptogramas $\{(y_i,c_i)\}_{i\in\{1,n\}}\,$
        
Então,  para todo $\,i\in I\,$,  pode recuperar  a mensagem
                                                        $$m_i \,\gets\, D_{s_i}(y_i,c_i,\tau)$$ 


Como na cifra vamos fazer com que a decifra possa receber a tag de autenticação:

In [28]:
def dec_fujisaki_OT(sk,pk,y,c,tau):
    r = int(dec(sk,pk,c))
    rlinha = h_OT(r,y,tau)
    if c != f_p_OT(pk,r,int.from_bytes(rlinha,"big")):
        print("má decifração")
        return None
    else:
        print("boa decifração")
        res = bytes(a ^^ b for a, b in zip(y, g(r)))
        return res

In [29]:
# Decifrar e verificar as mensagens
decrypted_messages = []
for msg_number, idx in receiver.e.items(): 
    if idx < len(ciphertext_vector):  # Verifica se o índice é válido
        print(f"\nNúmero da mensagem: {msg_number}")
        print(f"Índice no vetor ciphertext_vector: {idx}")
        print(f"Chave privada (sk): {receiver.s_values[msg_number - 1]}")  # Ajuste para índice base 0
        print(f"Chave pública (pk): {receiver.p[idx]}")
        print("len(ciphertext_vector): ",len(ciphertext_vector))
        # Verifica se a mensagem foi cifrada corretamente
        if ciphertext_vector[idx] is None:
            print("idx:",idx)
            print(f"Mensagem {idx} não foi cifrada corretamente.")
            continue

        y, c = ciphertext_vector[idx]
        sk = receiver.s_values[msg_number - 1] 
        decrypted_message = dec_fujisaki_OT(sk, receiver.p[idx], y, c, tau)

        if decrypted_message is not None:
            decrypted_int = int.from_bytes(decrypted_message, 'big')
            print(f"Mensagem decifrada: {decrypted_int}")

            # Compara com a mensagem original no Provider
            if decrypted_int == provider.messages[idx]:
                print("Decifração bem-sucedida! A mensagem decifrada corresponde à original.")
                decrypted_messages.append(decrypted_message)
            else:
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                print("Erro na decifração: A mensagem decifrada NÃO corresponde à original.")
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        else:
            print("Erro na decifração intencional.")


Número da mensagem: 1
Índice no vetor ciphertext_vector: 0
Chave privada (sk): 248492468099137661529400966240921513341
Chave pública (pk): (202004771659655246425480962907746731652771861232440020221403333286306538524764058200972421408051565526700181442665046792233704956746326748898448334124423600744173207955168716705072005822912942131469111784557507965201605586179572334718494540472542272267290751567188101609129169118922464729488280084792746625635580290773455581677881326043137, 603275853292820270902229293475194444727, 3, 89165199563069828886054116478886282170996939581769380580379423459937951132153850308044430380250510398721176792639613970374910386279898152096904237926790380805699651441347918946489794542260448262756209153500600148167413160450498333491128139414954106098790820889649643943979707694839319713055815021240405413159714985388130484581441039807814)
len(ciphertext_vector):  100
boa decifração
Mensagem decifrada: 1334516300730007610305256130224760598806598756949536275823211412419611